In [1]:
import psycopg2
import pandas as pd
import ast
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def connect_to_azure_postgres(host, database, user, password, port=5432):
    try:
        conn = psycopg2.connect(
            host=host,
            database=database,
            user=user,
            password=password,
            port=port
        )
        print("Kết nối thành công tới Azure PostgreSQL!")
        return conn
    except Exception as e:
        print("Không thể kết nối tới Azure PostgreSQL:")
        print(e)
        return None

In [3]:
def get_data():
    try:
        # Kết nối đến PostgreSQL
        host = "shopee.postgres.database.azure.com"
        database = "delivery_info"
        user = "Numpy"
        password = "!Namphuong592003"
        conn = connect_to_azure_postgres(host, database, user, password)
        query = """
            SELECT restaurant_id, menu
            FROM menu
            WHERE is_current = TRUE;
        """
        df = pd.read_sql_query(query, conn)
        return df

    except Exception as e:
        print("Không thể chèn dữ liệu vào Azure PostgreSQL:")
        print(e)
    finally:
        if conn:
            conn.close()

In [4]:
df_menu = get_data()

Kết nối thành công tới Azure PostgreSQL!


/tmp/ipykernel_7197/3720846325.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [ ]:
def parse_menu(menu):
    return pd.DataFrame(ast.literal_eval(menu))

In [15]:
temp = df_menu
menu_list = []
for _, row in temp.iterrows():
    restaurant_id = row['restaurant_id']
    menu_items = parse_menu(row['menu'])
    menu_list.append(menu_items)

menu_df = pd.concat(menu_list, ignore_index=True)

In [ ]:
# Chuẩn hóa tên món ăn
menu_df['name'] = menu_df['name'].str.strip().str.lower()


,name,price
0,bánh mì,5000
1,chanh dây,15000
2,olong,15000
3,coca,15000
4,nước suối,8000
...,...,...
2604714,watermelon+pineapple juice,40000
2604715,combo sức khoẻ: 1 bánh mì + 1 nước,22000
2604716,bánh mì hạt điều - chay,15000
2604717,nước sả tắc chia,10000


In [ ]:
# 1. Chuẩn bị dữ liệu text
names = menu_df['name'].values

# 2. Áp dụng TF-IDF
tfidf_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3))
tfidf_matrix = tfidf_vectorizer.fit_transform(names)

# Lấy từ điển các từ khóa và trọng số của chúng
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_scores = tfidf_matrix.sum(axis=0).A1  # Tổng trọng số của từng từ trong toàn bộ tập

# Tạo DataFrame để phân tích
tfidf_df = pd.DataFrame({
    "keyword": feature_names,
    "score": tfidf_scores
}).sort_values(by="score", ascending=False)

tfidf_df_filtered = tfidf_df[tfidf_df['keyword'].apply(lambda x: len(x.split()) >= 2)]
# Sắp xếp theo điểm số TF-IDF
tfidf_df_filtered = tfidf_df_filtered.sort_values(by="score", ascending=False)

# 4. In kết quả
print("Top từ khóa phổ biến (2 từ trở lên):")
print(tfidf_df_filtered.head(10))

Top từ khóa phổ biến nhất:
         keyword         score
1138186      trà  64461.827329
1044207      sữa  43913.982267
1146563  trà sữa  26902.833136
98214       bánh  26110.215011
820993      nước  24815.473226
329675       cơm  24553.927533
116724        bò  23524.379661
441559        gà  22488.173898
715478        mì  21691.497966
1339283       ép  19646.884473
